In [ ]:
import pandas as pd
import numpy as np
from numba import jit

In [ ]:
us = pd.read_csv('../covid-19-data/us.csv', parse_dates=['date'])

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (16,9)
plt.rcParams['figure.max_open_warning'] = 0

In [ ]:
dates = us['date']
us_daily_cases = us['cases'].diff().fillna(0)
us_daily_deaths = us['deaths'].diff().fillna(0)

In [ ]:
may31 = 496
dates[may31]

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.set_ylabel('daily cases (rolling 7-day average)')
ax2.set_ylabel('daily deaths (rolling 7-day average)')

p = ax1.plot(dates,us_daily_cases.rolling(7).mean(),color='orange')
p = ax2.plot(dates,us_daily_deaths.rolling(7).mean(), color='blue')

In [ ]:
from math import pi, exp, sqrt, log
from scipy.optimize import curve_fit

In [ ]:
@jit(nopython=True)
def gauss(x, mu, sigma):
    # normalized to unit integral
    norm = 1.0/(sigma * sqrt(2*pi))
    arg = -0.5 * ((x - mu)/sigma)**2
    return norm * exp(arg)

In [ ]:
@jit(nopython=True)
def normfn(i, norm1, norm1_end, norm2, norm2_end, norm3, norm3_end, norm4, norm4_end, norm5):
    if i < norm1_end:
#        return (norm2 - norm1)/norm1_end * i + norm1
        return norm1
    elif i < norm2_end:
        return (norm3 - norm2)/(norm2_end - norm1_end) * (i - norm1_end) + norm2
    elif i < norm3_end:
        return (norm4 - norm3)/(norm3_end - norm2_end) * (i - norm2_end) + norm3
    elif i < norm4_end:
        return (norm5 - norm4)/(norm4_end - norm3_end) * (i - norm3_end) + norm4
    else:
        return norm5

In [ ]:
@jit(nopython=True)
def full_model(newcases, norm1, norm1_end, norm2, norm2_end, norm3, norm3_end, norm4, norm4_end, norm5, offset, width):
    num = len(newcases)
    retval = np.zeros(num, np.float64)
    for i in range(0, num):
        for j in range(1, i):
            retval[i] += newcases[j] * normfn(j, norm1, norm1_end, norm2, norm2_end, norm3, norm3_end, norm4, norm4_end, norm5) * gauss(i-j, offset, width)
    return retval

In [ ]:
norm1_end = 75
norm2_end = 150
norm3_end = 200
norm4_end = may31
sigma = 18
@jit(nopython=True)
def model1(newcases, norm1, norm2, norm3, norm4, norm5, offset):
    global norm1_end, norm2_end, norm3_end, norm4_end, sigma
    return full_model(newcases, norm1, norm1_end, norm2, norm2_end, norm3, norm3_end, norm4, norm4_end, norm5, offset, sigma)

In [ ]:
def us_fit(dates, daily_cases, daily_deaths, plot=True):
    sigma = np.ones(len(daily_cases))
    sigma[0:150] = 10
    #popt, pcov = curve_fit(model1, dc, dd, [10, 0.1, 0.1, 0.002, 0.002, 18], sigma)
    popt, pcov = curve_fit(model1, daily_cases.to_numpy(), daily_deaths.to_numpy(),
        [10, 0.1, 0.1, 0.002, 0.002, 18], sigma)
    if plot:
        fig, ax = plt.subplots()
        ax.plot(dates, daily_deaths.rolling(7).mean(), color='blue', label='observed deaths')
        ax.plot(dates, model1(daily_cases.to_numpy(), *popt), color='red', label='model deaths')
        ax.axvspan(dates[0], dates[149], alpha=0.2)
    return popt, pcov

In [ ]:
len(us_daily_cases)

In [ ]:
fitend = 400
popt, pcov = us_fit(dates[:fitend], us_daily_cases[:fitend], us_daily_deaths[:fitend])

In [ ]:
popt

In [ ]:
perr = np.sqrt(np.diag(pcov))
perr

In [ ]:
plt.plot(dates, 
    [normfn(i, popt[0], norm1_end, popt[1], norm2_end, popt[2], norm3_end, popt[3], norm4_end, popt[4]) for i in range(0, len(dates))]);

In [ ]:
fig, ax = plt.subplots()
ax.plot(dates, us_daily_deaths.rolling(7).mean(), color='blue', label='observed deaths')
ax.plot(dates, model1(np.array(us_daily_cases), *popt), color='red', label='model deaths')
ax.axvspan(dates[0], dates[149], alpha=0.2)



In [ ]:
indices = range(0, len(dates))
past_days = 15
start_recent = len(dates) - past_days
recent_dates = dates[start_recent:]
recent_indices = indices[start_recent:]
recent_smooth_cases = us_daily_cases.rolling(7).mean()[start_recent:]
log_recent_smooth_cases = [log(x) for x in recent_smooth_cases]

In [ ]:
order = 2
coeffs, coeff_cov = np.polyfit(recent_indices, log_recent_smooth_cases, order, cov=True)
log_case_model = np.poly1d(coeffs)
last = recent_indices[-1]
future_num = 30
future_indices = range(last, last + future_num)

In [ ]:
print("polynomial coeffs", coeffs)
print("polynomial coeff errs", np.sqrt(np.diag(coeff_cov)))

In [ ]:
n = len(us_daily_cases)
N = n + future_num
future_dates = pd.date_range(dates[0], periods=N)
future_cases = np.zeros(N, np.float64)
future_cases[0:n] = us_daily_cases.rolling(7).mean().fillna(0)
future_cases[n:] =  [exp(log_case_model(i)) for i in future_indices]

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.set_ylabel('daily cases (rolling 7-day average)')
ax2.set_ylabel('daily deaths (rolling 7-day average)')

start = 250
p = ax1.plot(future_dates[n:], future_cases[n:], '.', color='orange', label='assumed future cases')
#p = ax1.plot(dates[start:], us_daily_cases.rolling(7).mean()[start:],color='blue', label='observed cases')
p = ax1.plot(dates[start:], us_daily_cases.rolling(7).mean()[start:],color='orange', label='observed cases')
p = ax2.plot(future_dates[start:], model1(future_cases, popt[0], popt[1], popt[2], popt[3], popt[4], popt[5])[start:], color='red', label='model deaths')
p = ax2.plot(dates[start:], us_daily_deaths.rolling(7).mean()[start:], color='blue', label='observed deaths')

yl1 = ax1.get_ylim()
ax1.set_ylim([0, yl1[1]])

yl2 = ax2.get_ylim()
ax2.set_ylim([0, yl2[1]])
fig.legend();

In [ ]:
states = pd.read_csv('../covid-19-data/us-states.csv', parse_dates=['date'])

In [ ]:
state_deaths = states.pivot(index='date', columns='state', values='deaths')
state_cases =  states.pivot(index='date', columns='state', values='cases')
all_states = states['state'].unique()
all_states.sort()

In [ ]:
norm1 = popt[0]
norm2 = popt[1]
offset = popt[-1]
@jit(nopython=True)
def model2(newcases, norm3, norm4, norm5):
    global norm1_end, norm2_end, norm3_end, norm4_end, sigma
    global norm1, norm2, offset
    return full_model(newcases, norm1, norm1_end, norm2, norm2_end, norm3, norm3_end, norm4, norm4_end, norm5, offset, sigma)
    

In [ ]:
def state_fit(state, state_cases, state_deaths, plot=True):
    daily_cases = state_cases[state].diff().fillna(0)
    daily_deaths = state_deaths[state].diff().fillna(0)
    sigma = np.ones(len(daily_cases))
    sigma[0:150] = 10
    popt, pcov = curve_fit(model2, daily_cases.to_numpy(), us_daily_deaths.to_numpy(),
            [0.002, 0.002, 0.002], sigma)
    if plot:
        fig, ax = plt.subplots()
        ax.plot(dates, daily_deaths.rolling(7).mean(), color='orange', label='observed deaths')
        ax.plot(dates, model2(daily_cases.to_numpy(), *popt), color='red', label='model deaths')
        ax.axvspan(dates[0], dates[149], alpha=0.2)
        plt.title("{state} fatality rate: {percentnorm:0.2f}%"
                  .format(state=state, percentnorm=100*popt[1]))
    print("{state} fatality rate: {percentnorm:0.2f}%"
          .format(state=state, percentnorm=100*popt[1]))
    return popt

In [ ]:
for state in all_states:
    state_fit(state, state_cases, state_deaths, False);

In [ ]:
def state_fit_full(state, state_cases, state_deaths, plot=True):
    daily_cases = state_cases[state].diff().fillna(0)
    dc = np.array(daily_cases)
    daily_deaths = state_deaths[state].diff().fillna(0)
    dd = np.array(daily_deaths)
    sigma = np.ones(len(daily_cases))
    sigma[0:150] = 10
    popt, pcov = curve_fit(model1, dc, dd, [10, 0.1, 0.1, 0.002, 0.002, 18], sigma)
    if plot:
        fig, ax = plt.subplots()
        ax.plot(dates, daily_deaths.rolling(7).mean(), color='orange', label='observed deaths')
        ax.plot(dates, model1(dc, *popt), color='red', label='model deaths')
        ax.axvspan(dates[0], dates[149], alpha=0.2)
        plt.title("{state} fatality rate: {percentnorm:0.2f}%"
                  .format(state=state, percentnorm=100*popt[4]))
    print("{state} fatality rate: {percentnorm:0.2f}%"
          .format(state=state, percentnorm=100*popt[4]))
    print("{state} fit: {popt}".format(state=state, popt=popt))
    return popt, pcov

In [ ]:
state_results = {}
for state in all_states:
    state_results[state] = state_fit_full(state, state_cases, state_deaths, True)

In [ ]:
state_results

In [ ]:
np.linalg.det(state_results['Illinois'][1])

In [ ]:
np.linalg.det(state_results['Michigan'][1])

In [ ]:
states = state_results.keys()
for state in states:
    p = state_results[state][0]
    ratio = p[4]/p[3]
    print(f"{state}: {ratio:0.2f}")